<a href="https://colab.research.google.com/github/AeroEng16/TennisPrediction/blob/main/playerDataScraping_ATP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re
import json
import datetime
import pandas as pd


In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}

apiLink = "https://atptour.com/en/-/www/rank/sglroll/200?v=1"

individualPlayerBaseURL = "https://www.atptour.com/en/-/www/stats/PLAYERID/YEAR/COURTTYPE?v=1"
individualPlayerBaseURL_Physicals = "https://www.atptour.com/en/-/www/players/hero/PLAYERID?v=1"



year="all"  #year (e.g. 2024) or all
courtType = "all" #grass,clay,hard, carpet or all
allCourts = ["Grass","Clay","Hard","all"]
playerDataSrc = requests.get(apiLink,headers=headers).json()



In [9]:
playerData = []
for i in playerDataSrc:


    filteredDataSet = {key: value for key, value in zip(i.keys(), i.values()) if "RankNo" in key or "Name" in key or "Points" in key or "PlayerId" in key or "Movement" in key}
    # Extract Player Physical/Ranking Stats
    localURL_Physicals = individualPlayerBaseURL_Physicals.replace("PLAYERID",filteredDataSet["PlayerId"])
    responseIndividualPlayer_Physicals = requests.get(localURL_Physicals,headers=headers).json()
    relevantPlayerStats = {key: value for key, value in zip(responseIndividualPlayer_Physicals.keys(), responseIndividualPlayer_Physicals.values()) if "Age" in key or "HeightCm" in key or "WeightKg" in key or "PlayHand" in key or "SglHiRank" in key or "SglCareerWon" in key or "SglCareerLost" in key or "SglYtdWon" in key or "SglYtdLost" in key or "SglHiRankDate" in key}
    relevantPlayerStats["PlayHand"] = relevantPlayerStats["PlayHand"]['Id']

    for k,j in enumerate(["PLAYERID","YEAR","COURTTYPE"]):
        if k==0:
            localURL = individualPlayerBaseURL.replace(j,filteredDataSet["PlayerId"])
        elif k == 1:
            localURL = localURL.replace(j,"all")
        elif k == 2:
            localURL = localURL.replace(j,courtType)

    responseIndividualPlayer = requests.get(localURL,headers=headers)

    firstStatYear = responseIndividualPlayer.json()["FirstStatYear"]
    lastStatYear  = responseIndividualPlayer.json()["LastStatYear"]
    firstStatYear = max(firstStatYear,2022)
    yearRange = [str(i) for i in range(firstStatYear,lastStatYear+1)]
    yearRange.append('all')

    for l in yearRange:
      for m in allCourts:
        for k,j in enumerate(["PLAYERID","YEAR","COURTTYPE"]):
          if k==0:
              localURL = individualPlayerBaseURL.replace(j,filteredDataSet["PlayerId"])
          elif k == 1:
              localURL = localURL.replace(j,str(l))
          elif k == 2:
              localURL = localURL.replace(j,m)
        responseIndividualPlayer = requests.get(localURL,headers=headers)

        servingStats = responseIndividualPlayer.json()["Stats"]["ServiceRecordStats"]
        recevingStats = responseIndividualPlayer.json()["Stats"]["ReturnRecordStats"]
        entryDetails = {"Year":l,"courtType":m}
        relevantServingStats = {key: value for key, value in zip(servingStats.keys(), servingStats.values()) if "Percentage" in key }
        relevantReceivingStats = {key: value for key, value in zip(recevingStats.keys(), recevingStats.values()) if "Percentage" in key }
        overallPlayerData = {**filteredDataSet,**entryDetails,**relevantPlayerStats,**relevantServingStats,**relevantReceivingStats}
        #print(json.dumps(overallplayerData, indent=2))
        playerData.append(overallPlayerData)


In [10]:
playerDF = pd.DataFrame(playerData)

todayDate = datetime.datetime.now().strftime("%x").replace("/","_")
playerDF.to_csv("playerData_Year_"+year+"_courtType_"+courtType+"_"+todayDate, sep=',', index=False, encoding='utf-8')

In [11]:
playerDF = pd.DataFrame(playerData)
playerDF.tail(50)

,RankNo,Name,Points,Movement,PlayerId,Year,courtType,Age,HeightCm,WeightKg,...,SecondServePointsWonPercentage,BreakPointsSavedPercentage,ServiceGamesWonPercentage,ServicePointsWonPercentage,FirstServeReturnPointsWonPercentage,SecondServeReturnPointsWonPercentage,BreakPointsConvertedPercentage,ReturnGamesWonPercentage,ReturnPointsWonPercentage,TotalPointsWonPercentage
3680,198,Valentin Royer,304,2,R0EB,2024,Grass,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3681,198,Valentin Royer,304,2,R0EB,2024,Clay,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3682,198,Valentin Royer,304,2,R0EB,2024,Hard,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3683,198,Valentin Royer,304,2,R0EB,2024,Carpet,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3684,198,Valentin Royer,304,2,R0EB,2024,all,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3685,198,Valentin Royer,304,2,R0EB,all,Grass,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3686,198,Valentin Royer,304,2,R0EB,all,Clay,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3687,198,Valentin Royer,304,2,R0EB,all,Hard,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3688,198,Valentin Royer,304,2,R0EB,all,Carpet,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3689,198,Valentin Royer,304,2,R0EB,all,all,23,188,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:

#iterate between current year (replace '&season=CURRENTSEASON' with blank) and 2022, collect dates for each year and output player elo ratings to a CSV for each date in each year
import requests
import datetime

seasonDatesURL = 'https://www.ultimatetennisstatistics.com/seasonRankingDates?rankType=ELO_RANK&season=CURRENTSEASON'
rankingInfoURL = 'https://www.ultimatetennisstatistics.com/rankingsTableTable?current=1&rowCount=500&sort%5Brank%5D=asc&searchPhrase=&rankType=ELO_RANK&season=CURRENTSEASON&date=CURRENTDATE&_=1716998656044'
df_playerELO = pd.DataFrame()

#for year in ['2022','2023','2024']:
for year in ['2022','2023']:

    if year == '2024':
        datesURL = seasonDatesURL.replace("&season=CURRENTSEASON","")
    else:
        datesURL = seasonDatesURL.replace("CURRENTSEASON",year)

    datesResponse = requests.get(datesURL)
    try:
        datesJSONresponse = datesResponse.json()
    except requests.exceptions.JSONDecodeError:
        print("jsonError")
        continue
    for date in datesJSONresponse:
        rankingURL = rankingInfoURL.replace('CURRENTSEASON',year)
        formattedDate = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('%d-%m-%Y')

        rankingURL = rankingURL.replace('CURRENTDATE',formattedDate)
        rankingResponse = requests.get(rankingURL)
        try:
            rankingJSONResponse = rankingResponse.json()
        except requests.exceptions.JSONDecodeError:
            print("jsonError")
            continue


        reff = pd.json_normalize(rankingJSONResponse["rows"])
        df = pd.DataFrame(data=reff)
        df["RankingDate"] = formattedDate
        df = df[["rank","name","points","bestPoints","RankingDate"]]

        df_playerELO = pd.concat([df_playerELO, df], ignore_index=True)

df_playerELO.head()

,rank,name,points,bestPoints,RankingDate
0,1,Novak Djokovic,2417,2629,04-12-2022
1,2,Rafael Nadal,2311,2552,04-12-2022
2,3,Alexander Zverev,2237,2303,04-12-2022
3,4,Carlos Alcaraz Garfia,2191,2344,04-12-2022
4,5,Daniil Medvedev,2185,2316,04-12-2022


In [12]:
df_playerELO.to_csv("eloRankings.csv", sep=',', index=False, encoding='utf-8')